In [1]:
# Install required packages
! pip install crewai pyboxen langchain langchain_community langchain_openai python-dotenv PyPDF2 agentops openai

In [2]:
#  Import libraries and mount Google Drive

# Import necessary libraries
from dotenv import load_dotenv
import os
from crewai import Crew, Agent
from pyboxen import boxen
import agentops
from langchain.chat_models import ChatOpenAI
# Initialize environment variables
load_dotenv()

True

In [4]:
#  Initialize AgentOps and API key
agentops.init(tags=['contract-analysis'])
# Check if OPENAI_API_KEY is set in .env file and load it if not already set
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
  from dotenv import load_dotenv
  load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

🖇 AgentOps: The 'tags' parameter is deprecated. Use 'default_tags' instead
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=93190b46-ddb9-4298-b4a8-1a8a4ad2bb49


In [5]:
#  Let's create a list of important contract rules! 📋

reference_data = {
    "limitation of liability": "30 days",      # ⚖️ How long you can report problems
    "owner expiry date": "30 days",            # 📅 When owner rights end
    "notice period": "30 days",                # 📣 Time needed to tell about changes
    "agreement duration": "12 months",         # 📆 How long the contract lasts
    "payment terms": "30 days",                # 💰 When you need to pay by
    "confidentiality period": "3 years",       # 🤫 How long to keep secrets
    "insurance coverage": "$1,000,000",        # 🛡️ How much insurance protection
    "maximum monthly hours": "120"             # ⏰ Most hours allowed per month
}

# This part makes a pretty blue box to show all our rules nicely!
print(boxen(
    "\n".join([f"{k}: {v}" for k, v in reference_data.items()]),
    title="📋 Reference Contract Terms",
    padding=1,
    margin=1,
    color="blue"
))

                                                                                                                   
   ╭─ 📋 Reference Contract Terms ────────╮                                                                        
   │                                      │                                                                        
   │   limitation of liability: 30 days   │                                                                        
   │   owner expiry date: 30 days         │                                                                        
   │   notice period: 30 days             │                                                                        
   │   agreement duration: 12 months      │                                                                        
   │   payment terms: 30 days             │                                                                        
   │   confidentiality period: 3 years    │                              

In [6]:
# Tool to read and extract text from PDF files
from langchain.tools import tool
from PyPDF2 import PdfReader
from typing import Optional

@tool
def pdf_reader(file_path: str) -> str:
    """Read and extract text from PDF files"""
    try:
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def reference_data_tool(query: Optional[str] = None) -> dict:
    """Access the standard reference data for comparison"""
    return reference_data

In [7]:
# Crew Ai Agents Memory Example

from rich import box
from rich.console import Console
from rich.table import Table
from rich.syntax import Syntax

def create_tables():
    console = Console()

    # Components Table
    components_table = Table(box=box.DOUBLE, title="Memory System Components",
                           show_header=True, header_style="bold cyan", show_lines=True)

    components_table.add_column("Component", style="bold black")
    components_table.add_column("Description", style="dim black")

    components = [
        ("Short-Term Memory", "Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall and utilize information relevant to their current context during executions."),
        ("Long-Term Memory", "Preserves valuable insights and learnings from past executions, allowing agents to build and refine their knowledge over time."),
        ("Entity Memory", "Captures and organizes information about entities (people, places, concepts) encountered during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for storing entity information."),
        ("Contextual Memory", "Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and EntityMemory, aiding in coherence and relevance of agent responses."),
        ("User Memory", "Stores user-specific information and preferences, enhancing personalization and user experience.")
    ]

    for component, desc in components:
        components_table.add_row(component, desc)

    console.print(components_table)

    # Usage Example Table
    example_table = Table(box=box.DOUBLE, title="Code Snippet Using Memory System Usage Example with crew ai agents",
                         show_header=True, header_style="bold cyan", show_lines=True)

    example_table.add_column("Code Example", style="bold black")

    example_code = '''
from crewai import Crew, Agent, Task, Process

# Assemble your crew with memory capabilities

# Long-term memory - Stores persistent data in SQLite
my_crew = Crew(
    agents=[...],
    tasks=[...],
    process="Process.sequential",
    memory=True,
    long_term_memory=EnhanceLongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="/my_data_dir/my_crew1/long_term_memory_storage.db"
        )
    ),

    # Short-term memory - Handles recent context using RAG
    short_term_memory=EnhanceShortTermMemory(
        storage=CustomRAGStorage(
            crew_name="my_crew",
            storage_type="short_term",
            data_dir="//my_data_dir",
            model=embedder["model"],
            dimension=embedder["dimension"],
        ),
    ),

    # Entity memory - Tracks information about specific entities
    entity_memory=EnhanceEntityMemory(
        storage=CustomRAGStorage(
            crew_name="my_crew",
            storage_type="entities",
            data_dir="//my_data_dir",
            model=embedder["model"],
            dimension=embedder["dimension"],
        ),
    ),
    verbose=True,
)
'''

    example_table.add_row(example_code)

    # Print tables with spacing
    console.print("\n")
    console.print(example_table)

if __name__ == "__main__":
    create_tables()

                                             Memory System Components                                              
╔═══════════════════╦═════════════════════════════════════════════════════════════════════════════════════════════╗
║ Component         ║ Description                                                                                 ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Short-Term Memory ║ Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall    ║
║                   ║ and utilize information relevant to their current context during executions.                ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Long-Term Memory  ║ Preserves valuable insights and learnings from past executions, allowing agents to build    ║
║                   ║ and refine their knowledge over time.                                                       ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Entity Memory     ║ Captures and organizes information about entities (people, places, concepts) encountered    ║
║                   ║ during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for      ║
║                   ║ storing entity information.                                                                 ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Contextual Memory ║ Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and     ║
║                   ║ EntityMemory, aiding in coherence and relevance of agent responses.                         ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ User Memory       ║ Stores user-specific information and preferences, enhancing personalization and user        ║
║                   ║ experience.                                                                                 ║
╚═══════════════════╩═════════════════════════════════════════════════════════════════════════════════════════════╝

    Code Snippet Using Memory System Usage Example with crew ai agents     
╔═════════════════════════════════════════════════════════════════════════╗
║ Code Example                                                            ║
╠═════════════════════════════════════════════════════════════════════════╣
║                                                                         ║
║ from crewai import Crew, Agent, Task, Process                           ║
║                                                                         ║
║ # Assemble your crew with memory capabilities                           ║
║                                                                         ║
║ # Long-term memory - Stores persistent data in SQLite                   ║
║ my_crew = Crew(                                                         ║
║     agents=[...],                                                       ║
║     tasks=[...],                                                        ║
║     process="Process.sequential",                                       ║
║     memory=True,                                                        ║
║     long_term_memory=EnhanceLongTermMemory(                             ║
║         storage=LTMSQLiteStorage(                                       ║
║             db_path="/my_data_dir/my_crew1/long_term_memory_storage.db" ║
║         )                                                               ║
║     ),                                                                  ║
║                                                                         ║
║     # Short-term memory - Handles recent context using RAG              ║
║     short_term_memory=EnhanceShortTermMemory(                           ║
║         storage=CustomRAGStorage(                                       ║
║             crew_name="my_crew",                                        ║
║             storage_type="short_term",                                  ║
║             data_dir="//my_data_dir",                                   ║
║             model=embedder["model"],                                    ║
║             dimension=embedder["dimension"],                            ║
║         ),                                                              ║
║     ),                                                                  ║
║                                                                         ║
║     # Entity memory - Tracks information about specific entities        ║
║     entity_memory=EnhanceEntityMemory(                                  ║
║         storage=CustomRAGStorage(                                       ║
║             crew_name="my_crew",                                        ║
║             storage_type="entities",                                    ║
║             data_dir="//my_data_dir",                                   ║
║             model=embedder["model"],                                    ║
║             dimension=embedder["dimension"],                            ║
║         ),                                                              ║
║     ),                                                                  ║
║     verbose=True,                                                       ║
║ )                                                                       ║
║                                                                         ║
╚═════════════════════════════════════════════════════════════════════════╝

In [8]:
# Crew Ai Agents
from langchain_openai import ChatOpenAI

def create_agents():
    researcher_agent = Agent(
        role='PDF Data Extractor',
        goal='Extract key contract terms from PDF',
        backstory='Expert in extracting contract terms from PDFs',
        llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
        verbose=True,
        tools=[pdf_reader],
        memory=True,
    )

    writer_agent = Agent(
        role='Data Comparison Analyst',
        goal='Compare contract terms with reference values',
        backstory='Expert in analyzing contract terms',
        llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
        verbose=True,
        tools=[reference_data_tool],
        memory=True,
    )

    review_agent = Agent(
        role='Compliance Report Generator',
        goal='Create detailed report of contract deviations',
        backstory='Expert in compliance reporting',
        llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
        verbose=True,
        tools=[reference_data_tool],
        memory=True,
    )

    for agent in [researcher_agent, writer_agent, review_agent]:
        print(boxen(
            f"Role: {agent.role}\nGoal: {agent.goal}\nBackstory: {agent.backstory}",
            title="🤖 Agent Profile",
            padding=1,
            margin=1,
            color="green"
        ))

    return researcher_agent, writer_agent, review_agent

# Initialize agents
researcher_agent, writer_agent, review_agent = create_agents()

                                                                                                                   
   ╭─ 🤖 Agent Profile ───────────────────────────────────────────╮                                                
   │                                                              │                                                
   │   Role: PDF Data Extractor                                   │                                                
   │   Goal: Extract key contract terms from PDF                  │                                                
   │   Backstory: Expert in extracting contract terms from PDFs   │                                                
   │                                                              │                                                
   ╰──────────────────────────────────────────────────────────────╯                                                
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ─────────────────────────────────────╮                                                      
   │                                                        │                                                      
   │   Role: Data Comparison Analyst                        │                                                      
   │   Goal: Compare contract terms with reference values   │                                                      
   │   Backstory: Expert in analyzing contract terms        │                                                      
   │                                                        │                                                      
   ╰────────────────────────────────────────────────────────╯                                                      
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ──────────────────────────────────────╮                                                     
   │                                                         │                                                     
   │   Role: Compliance Report Generator                     │                                                     
   │   Goal: Create detailed report of contract deviations   │                                                     
   │   Backstory: Expert in compliance reporting             │                                                     
   │                                                         │                                                     
   ╰─────────────────────────────────────────────────────────╯                                                     
                                                                         

In [9]:
# Crew Ai Agents Tasks

from crewai import Task
from textwrap import dedent

def create_tasks(researcher_agent, writer_agent, review_agent, input_file):
    pdf_extraction_task = Task(
        description=dedent(f"""Use the PDF Reader tool to read {input_file} and extract key terms."""),
        agent=researcher_agent,
        expected_output="Dictionary containing extracted contract terms"
    )

    comparison_task = Task(
        description=dedent(f"""Compare extracted terms with reference data and identify mismatches."""),
        agent=writer_agent,
        expected_output="List of mismatches between extracted and reference data"
    )

    final_report_task = Task(
        description=dedent(f"""Create comprehensive analysis report with findings."""),
        agent=review_agent,
        expected_output="Detailed analysis report with findings and recommendations"
    )

    for task in [pdf_extraction_task, comparison_task, final_report_task]:
        print(boxen(
            f"Description: {task.description}\nExpected Output: {task.expected_output}",
            title="📝 Task Details",
            padding=1,
            margin=1,
            color="yellow"
        ))

    return pdf_extraction_task, comparison_task, final_report_task

In [14]:
import requests
import os

def download_github_pdf():
    # Convert GitHub URL to raw content URL
    github_url = "https://github.com/initmahesh/MLAI-community-labs/blob/main/Class-Labs/Lab-5(CrewAIMulti-Agent-Document-Analysis)/contract.pdf"
    raw_url = github_url.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")

    # Create directory if it doesn't exist
    save_path = "/Users/pranavgupta/Desktop/Crew AI Multi Agent"

    try:
        # Download the file
        response = requests.get(raw_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Save the file to the specified directory
        file_path = os.path.join(save_path, "contract.pdf")
        with open(file_path, 'wb') as f:
            f.write(response.content)

        print(f"Successfully downloaded and saved to: {file_path}")
        return file_path

    except Exception as e:
        print(f"Error downloading file: {str(e)}")
        return None

# Download the file and get the path
input_file = download_github_pdf()

if input_file:
    # Create tasks with the new file path
    tasks = create_tasks(researcher_agent, writer_agent, review_agent, input_file)

    def run_analysis(agents, tasks):
        # Remove any leading/trailing whitespaces or newline characters from the API key
        os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY").strip()

        crew = Crew(
            agents=agents,
            tasks=tasks,
            verbose=True
        )

        result = crew.kickoff()

        print(boxen(
            str(result),
            title="📊 Contract Analysis Report",
            padding=1,
            margin=1,
            color="magenta"
        ))

        return result

    # Execute analysis
    final_report = run_analysis([researcher_agent, writer_agent, review_agent], tasks)
else:
    print("Could not proceed with analysis due to file download error")

Successfully downloaded and saved to: /Users/pranavgupta/Desktop/Crew AI Multi Agent/contract.pdf


                                                                                                                   
   ╭─ 📝 Task Details ─────────────────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │          Description: Use the PDF Reader tool to read /Users/pranavgupta/Desktop/Crew AI Multi            │   
   │          Agent/contract.pdf and extract key terms.                                                        │   
   │          Expected Output: Dictionary containing extracted contract terms                                  │   
   │                                                                                                           │   
   ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯   
                                                                         

                                                                                                                   
   ╭─ 📝 Task Details ─────────────────────────────────────────────────────────────────────╮                       
   │                                                                                       │                       
   │   Description: Compare extracted terms with reference data and identify mismatches.   │                       
   │   Expected Output: List of mismatches between extracted and reference data            │                       
   │                                                                                       │                       
   ╰───────────────────────────────────────────────────────────────────────────────────────╯                       
                                                                                                                   



                                                                                                                   
   ╭─ 📝 Task Details ───────────────────────────────────────────────────────────────╮                             
   │                                                                                 │                             
   │   Description: Create comprehensive analysis report with findings.              │                             
   │   Expected Output: Detailed analysis report with findings and recommendations   │                             
   │                                                                                 │                             
   ╰─────────────────────────────────────────────────────────────────────────────────╯                             
                                                                                                                   

# Agent: PDF Data Extractor
## Task: Use the PDF Reader tool to read /Us

🖇 AgentOps: Session Stats - Duration: 8m 37.7s | Cost: $0.297120 | LLMs: 9 | Tools: 5 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=93190b46-ddb9-4298-b4a8-1a8a4ad2bb49


                                                                                                                   
   ╭─ 📊 Contract Analysis Report ─────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Detailed Analysis Report                                                                                │   
   │                                                                                                           │   
   │   1. Limitation of Liability: The contract states that the Consultant's liability is limited to 45 days   │   
   │   of fees paid, while the standard reference data suggests a limit of 30 days. This is a deviation of     │   
   │   15 days.                                                                                                │   
   │                                                                     